# Pré-processamento dos Datasets para Experimento RCBD

**Equipe F**: Bernardo Bacha de Resende, Gustavo Augusto Faria dos Reis, Marília Macêdo de Melo

**Disciplina**: EEE933 - Planejamento e Análise de Experimentos (2025/2)

---

Este notebook processa os 5 datasets de classificação binária, preparando-os para uso no experimento RCBD.

**Datasets:**
1. Breast Cancer (569 amostras)
2. Titanic (891 amostras)
3. Water Potability (3,276 amostras)
4. Employee Attrition (4,653 amostras)
5. Australia Rain (145,460 amostras → amostragem para ~10k)

**Pré-processamento aplicado:**
- Remoção de colunas não informativas (IDs, nomes, etc.)
- Tratamento de valores nulos (mediana para numérico, moda para categórico)
- One-hot encoding para features categóricas
- StandardScaler (z-score) para normalização

## 1. Imports e Configurações

In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

print("✓ Bibliotecas importadas com sucesso!")

✓ Bibliotecas importadas com sucesso!


In [23]:
# Instalar scikit-optimize se necessário
import sys
import subprocess

try:
    import skopt
    print("✓ scikit-optimize já instalado")
except ImportError:
    print("Instalando scikit-optimize...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "scikit-optimize"])
    print("✓ scikit-optimize instalado com sucesso!")

✓ scikit-optimize já instalado


## 2. Dataset 1: Breast Cancer

**Características:**
- 569 amostras × 32 colunas
- Target: `diagnosis` (M=Malignant, B=Benign)
- Sem valores nulos
- Dataset mais limpo

In [24]:
print("=" * 80)
print("DATASET 1: BREAST CANCER")
print("=" * 80)

# Carregar dataset
df_breast = pd.read_csv('../data/breast_cancer.csv')
print(f"Shape original: {df_breast.shape}")
print(f"Colunas: {list(df_breast.columns)}")

# Remover coluna ID
df_breast = df_breast.drop(columns=['id'])

# Separar target
y_breast_cancer = df_breast['diagnosis'].map({'M': 1, 'B': 0})
X_breast_cancer = df_breast.drop(columns=['diagnosis'])

# Verificar nulos
print(f"\nValores nulos: {X_breast_cancer.isnull().sum().sum()}")

# Normalizar com StandardScaler
scaler_breast = StandardScaler()
X_breast_cancer = pd.DataFrame(
    scaler_breast.fit_transform(X_breast_cancer),
    columns=X_breast_cancer.columns
)

print(f"\n✓ Pré-processamento concluído!")
print(f"X_breast_cancer shape: {X_breast_cancer.shape}")
print(f"y_breast_cancer shape: {y_breast_cancer.shape}")
print(f"Distribuição do target:\n{y_breast_cancer.value_counts()}")
print(f"Proporção: {y_breast_cancer.value_counts(normalize=True)}")

DATASET 1: BREAST CANCER
Shape original: (569, 32)
Colunas: ['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']

Valores nulos: 0

✓ Pré-processamento concluído!
X_breast_cancer shape: (569, 30)
y_breast_cancer shape: (569,)
Distribuição do target:
diagnosis
0    357
1    212
Name: count, dtype: int64
Proporção: diagnosis
0    0.627417
1    0.372583
Name: proportion, dtype: float64


## 3. Dataset 2: Titanic

**Características:**
- 891 amostras × 12 colunas
- Target: `Survived` (0/1)
- Valores nulos em Age, Cabin, Embarked
- Features categóricas: Sex, Embarked, Pclass

In [25]:
print("=" * 80)
print("DATASET 2: TITANIC")
print("=" * 80)

# Carregar dataset
df_titanic = pd.read_csv('../data/titanic.csv')
print(f"Shape original: {df_titanic.shape}")
print(f"\nValores nulos por coluna:\n{df_titanic.isnull().sum()[df_titanic.isnull().sum() > 0]}")

# Remover colunas não informativas
df_titanic = df_titanic.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

# Separar target
y_titanic = df_titanic['Survived']
X_titanic = df_titanic.drop(columns=['Survived'])

# Tratar valores nulos
# Age: preencher com mediana
X_titanic['Age'].fillna(X_titanic['Age'].median(), inplace=True)
# Embarked: preencher com moda
X_titanic['Embarked'].fillna(X_titanic['Embarked'].mode()[0], inplace=True)

print(f"\nValores nulos após tratamento: {X_titanic.isnull().sum().sum()}")

# One-hot encoding para categóricas
categorical_cols = ['Sex', 'Embarked']
X_titanic = pd.get_dummies(X_titanic, columns=categorical_cols, drop_first=True, dtype=int)

# Converter Pclass para dummy se ainda não for numérico adequado
# Pclass já é numérico (1, 2, 3), mas pode fazer one-hot se preferir
X_titanic = pd.get_dummies(X_titanic, columns=['Pclass'], prefix='Pclass', dtype=int)

print(f"\nColunas após one-hot encoding ({len(X_titanic.columns)}): {list(X_titanic.columns)}")

# Normalizar com StandardScaler
scaler_titanic = StandardScaler()
X_titanic = pd.DataFrame(
    scaler_titanic.fit_transform(X_titanic),
    columns=X_titanic.columns
)

print(f"\n✓ Pré-processamento concluído!")
print(f"X_titanic shape: {X_titanic.shape}")
print(f"y_titanic shape: {y_titanic.shape}")
print(f"Distribuição do target:\n{y_titanic.value_counts()}")
print(f"Proporção: {y_titanic.value_counts(normalize=True)}")

DATASET 2: TITANIC
Shape original: (891, 12)

Valores nulos por coluna:
Age         177
Cabin       687
Embarked      2
dtype: int64

Valores nulos após tratamento: 0

Colunas após one-hot encoding (10): ['Age', 'SibSp', 'Parch', 'Fare', 'Sex_male', 'Embarked_Q', 'Embarked_S', 'Pclass_1', 'Pclass_2', 'Pclass_3']

✓ Pré-processamento concluído!
X_titanic shape: (891, 10)
y_titanic shape: (891,)
Distribuição do target:
Survived
0    549
1    342
Name: count, dtype: int64
Proporção: Survived
0    0.616162
1    0.383838
Name: proportion, dtype: float64


## 4. Dataset 3: Water Potability

**Características:**
- 3,276 amostras × 10 colunas
- Target: `Potability` (0/1)
- Valores nulos em pH, Sulfate, Trihalomethanes
- Todas features numéricas

In [26]:
print("=" * 80)
print("DATASET 3: WATER POTABILITY")
print("=" * 80)

# Carregar dataset
df_water = pd.read_csv('../data/water_potability.csv')
print(f"Shape original: {df_water.shape}")
print(f"\nValores nulos por coluna:\n{df_water.isnull().sum()[df_water.isnull().sum() > 0]}")

# Separar target
y_water_potability = df_water['Potability']
X_water_potability = df_water.drop(columns=['Potability'])

# Tratar valores nulos (preencher com mediana)
for col in X_water_potability.columns:
    if X_water_potability[col].isnull().sum() > 0:
        X_water_potability[col].fillna(X_water_potability[col].median(), inplace=True)
        print(f"Preenchidos {col} com mediana: {X_water_potability[col].median():.2f}")

print(f"\nValores nulos após tratamento: {X_water_potability.isnull().sum().sum()}")

# Normalizar com StandardScaler
scaler_water = StandardScaler()
X_water_potability = pd.DataFrame(
    scaler_water.fit_transform(X_water_potability),
    columns=X_water_potability.columns
)

print(f"\n✓ Pré-processamento concluído!")
print(f"X_water_potability shape: {X_water_potability.shape}")
print(f"y_water_potability shape: {y_water_potability.shape}")
print(f"Distribuição do target:\n{y_water_potability.value_counts()}")
print(f"Proporção: {y_water_potability.value_counts(normalize=True)}")

DATASET 3: WATER POTABILITY
Shape original: (3276, 10)

Valores nulos por coluna:
ph                 491
Sulfate            781
Trihalomethanes    162
dtype: int64
Preenchidos ph com mediana: 7.04
Preenchidos Sulfate com mediana: 333.07
Preenchidos Trihalomethanes com mediana: 66.62

Valores nulos após tratamento: 0

✓ Pré-processamento concluído!
X_water_potability shape: (3276, 9)
y_water_potability shape: (3276,)
Distribuição do target:
Potability
0    1998
1    1278
Name: count, dtype: int64
Proporção: Potability
0    0.60989
1    0.39011
Name: proportion, dtype: float64


## 5. Dataset 4: Employee Attrition

**Características:**
- 4,653 amostras × 9 colunas
- Target: `LeaveOrNot` (0=Ficou, 1=Saiu do emprego)
- Sem valores nulos
- Features categóricas: Education, City, Gender, EverBenched
- **Atenção**: Classes razoavelmente balanceadas (~34% saídas)

In [27]:
print("=" * 80)
print("DATASET 4: EMPLOYEE ATTRITION")
print("=" * 80)

# Carregar dataset
df_employee = pd.read_csv('../data/Employee.csv')
print(f"Shape original: {df_employee.shape}")
print(f"\nValores nulos por coluna:\n{df_employee.isnull().sum().sum()} (nenhum!)")

# Separar target
y_employee = df_employee['LeaveOrNot']
X_employee = df_employee.drop(columns=['LeaveOrNot'])

print(f"\nValores nulos: {X_employee.isnull().sum().sum()}")

# Identificar e fazer one-hot encoding para categóricas
categorical_cols = ['Education', 'City', 'Gender', 'EverBenched']
print(f"\nColunas categóricas para encoding: {categorical_cols}")

X_employee = pd.get_dummies(X_employee, columns=categorical_cols, drop_first=True, dtype=int)

print(f"\nColunas após one-hot encoding ({len(X_employee.columns)}): {list(X_employee.columns)}")

# Normalizar com StandardScaler
scaler_employee = StandardScaler()
X_employee = pd.DataFrame(
    scaler_employee.fit_transform(X_employee),
    columns=X_employee.columns
)

print(f"\n✓ Pré-processamento concluído!")
print(f"X_employee shape: {X_employee.shape}")
print(f"y_employee shape: {y_employee.shape}")
print(f"Distribuição do target:\n{y_employee.value_counts()}")
print(f"Proporção: {y_employee.value_counts(normalize=True)}")

DATASET 4: EMPLOYEE ATTRITION
Shape original: (4653, 9)

Valores nulos por coluna:
0 (nenhum!)

Valores nulos: 0

Colunas categóricas para encoding: ['Education', 'City', 'Gender', 'EverBenched']

Colunas após one-hot encoding (10): ['JoiningYear', 'PaymentTier', 'Age', 'ExperienceInCurrentDomain', 'Education_Masters', 'Education_PHD', 'City_New Delhi', 'City_Pune', 'Gender_Male', 'EverBenched_Yes']

✓ Pré-processamento concluído!
X_employee shape: (4653, 10)
y_employee shape: (4653,)
Distribuição do target:
LeaveOrNot
0    3053
1    1600
Name: count, dtype: int64
Proporção: LeaveOrNot
0    0.656136
1    0.343864
Name: proportion, dtype: float64


## 6. Dataset 5: Australia Rain (Weather)

**Características:**
- 145,460 amostras × 23 colunas (MUITO GRANDE)
- Target: `RainTomorrow` (Yes/No)
- MUITOS valores nulos (~40% das features)
- Features categóricas: WindGustDir, WindDir9am, WindDir3pm, RainToday

**Estratégia:**
1. Remover Date e Location (não informativas/muitas categorias)
2. Remover linhas com muitos nulos (dropna)
3. Fazer amostragem estratificada para ~10k amostras

In [28]:
print("=" * 80)
print("DATASET 5: AUSTRALIA RAIN (WEATHER)")
print("=" * 80)

# Carregar dataset
df_weather = pd.read_csv('../data/weather.csv')
print(f"Shape original: {df_weather.shape}")
print(f"\nValores nulos por coluna:")
null_counts = df_weather.isnull().sum()
print(null_counts[null_counts > 0])
print(f"\nTotal de colunas com nulos: {(null_counts > 0).sum()}")

# Remover colunas não informativas
df_weather = df_weather.drop(columns=['Date', 'Location'])
print(f"\nShape após remoção de Date e Location: {df_weather.shape}")

# Remover target nulo primeiro
df_weather = df_weather.dropna(subset=['RainTomorrow'])
print(f"Shape após remover target nulo: {df_weather.shape}")

# Separar target
y_weather = df_weather['RainTomorrow'].map({'Yes': 1, 'No': 0})
X_weather = df_weather.drop(columns=['RainTomorrow'])

# Remover linhas com muitos nulos (estratégia: dropna)
# Vamos remover linhas que têm qualquer valor nulo
initial_rows = len(X_weather)
valid_indices = X_weather.dropna().index
X_weather = X_weather.loc[valid_indices]
y_weather = y_weather.loc[valid_indices]

print(f"\nLinhas removidas por nulos: {initial_rows - len(X_weather)}")
print(f"Shape após dropna: X={X_weather.shape}, y={y_weather.shape}")

# One-hot encoding para categóricas
# Converter RainToday para numérico antes
if 'RainToday' in X_weather.columns:
    X_weather['RainToday'] = X_weather['RainToday'].map({'Yes': 1, 'No': 0})

# Identificar colunas categóricas (direções de vento)
categorical_cols = [col for col in X_weather.columns if 'WindDir' in col or 'Wind' in col and X_weather[col].dtype == 'object']
print(f"\nColunas categóricas identificadas: {categorical_cols}")

if categorical_cols:
    X_weather = pd.get_dummies(X_weather, columns=categorical_cols, drop_first=True, dtype=int)
    print(f"Colunas após one-hot encoding: {len(X_weather.columns)}")

# AMOSTRAGEM ESTRATIFICADA para ~10k amostras
if len(X_weather) > 10000:
    sample_size = 10000
    print(f"\nFazendo amostragem estratificada de {len(X_weather)} para {sample_size} amostras...")
    
    from sklearn.model_selection import train_test_split
    X_weather, _, y_weather, _ = train_test_split(
        X_weather, y_weather, 
        train_size=sample_size, 
        stratify=y_weather,
        random_state=42
    )
    print(f"Shape após amostragem: X={X_weather.shape}, y={y_weather.shape}")

# Normalizar com StandardScaler
scaler_weather = StandardScaler()
X_weather = pd.DataFrame(
    scaler_weather.fit_transform(X_weather),
    columns=X_weather.columns
)

print(f"\n✓ Pré-processamento concluído!")
print(f"X_weather shape: {X_weather.shape}")
print(f"y_weather shape: {y_weather.shape}")
print(f"Distribuição do target:\n{y_weather.value_counts()}")
print(f"Proporção: {y_weather.value_counts(normalize=True)}")

DATASET 5: AUSTRALIA RAIN (WEATHER)
Shape original: (145460, 23)

Valores nulos por coluna:
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

Total de colunas com nulos: 21

Shape após remoção de Date e Location: (145460, 21)
Shape após remover target nulo: (142193, 21)

Linhas removidas por nulos: 85773
Shape após dropna: X=(56420, 20), y=(56420,)

Colunas categóricas identificadas: ['WindGustDir', 'WindDir9am', 'WindDir3pm']
Colunas após one-hot encoding: 62

Fazendo amostragem estratificada de 56420 para 10000 amostras...
Shape após amostragem:

## 7. Resumo Final

Validação de todos os datasets processados

In [29]:
print("=" * 80)
print("RESUMO FINAL - TODOS OS DATASETS PROCESSADOS")
print("=" * 80)

datasets_summary = {
    'Breast Cancer': (X_breast_cancer, y_breast_cancer),
    'Titanic': (X_titanic, y_titanic),
    'Water Potability': (X_water_potability, y_water_potability),
    'Employee': (X_employee, y_employee),
    'Weather': (X_weather, y_weather)
}

summary_data = []

for name, (X, y) in datasets_summary.items():
    summary_data.append({
        'Dataset': name,
        'Amostras': X.shape[0],
        'Features': X.shape[1],
        'Nulos em X': X.isnull().sum().sum(),
        'Nulos em y': y.isnull().sum(),
        'Classe 0': (y == 0).sum(),
        'Classe 1': (y == 1).sum(),
        'Proporção (%)': f"{(y == 1).sum() / len(y) * 100:.1f}%"
    })

summary_df = pd.DataFrame(summary_data)
print("\n")
print(summary_df.to_string(index=False))

print("\n" + "=" * 80)
print("✓ TODOS OS DATASETS PRONTOS PARA USO NO EXPERIMENTO RCBD!")
print("=" * 80)

print("\n📊 Variáveis disponíveis:")
print("  • X_breast_cancer, y_breast_cancer")
print("  • X_titanic, y_titanic")
print("  • X_water_potability, y_water_potability")
print("  • X_employee, y_employee")
print("  • X_weather, y_weather")

print("\n🎯 Características:")
print("  • Todas as features são numéricas")
print("  • Sem valores nulos")
print("  • Normalizadas com StandardScaler (z-score)")
print("  • Prontas para classificadores de ML")

RESUMO FINAL - TODOS OS DATASETS PROCESSADOS


         Dataset  Amostras  Features  Nulos em X  Nulos em y  Classe 0  Classe 1 Proporção (%)
   Breast Cancer       569        30           0           0       357       212         37.3%
         Titanic       891        10           0           0       549       342         38.4%
Water Potability      3276         9           0           0      1998      1278         39.0%
        Employee      4653        10           0           0      3053      1600         34.4%
         Weather     10000        62           0           0      7797      2203         22.0%

✓ TODOS OS DATASETS PRONTOS PARA USO NO EXPERIMENTO RCBD!

📊 Variáveis disponíveis:
  • X_breast_cancer, y_breast_cancer
  • X_titanic, y_titanic
  • X_water_potability, y_water_potability
  • X_employee, y_employee
  • X_weather, y_weather

🎯 Características:
  • Todas as features são numéricas
  • Sem valores nulos
  • Normalizadas com StandardScaler (z-score)
  • Prontas para 

## 8. Baseline - SVM com Kernel RBF

**Objetivo:** Treinar SVM simples em cada dataset para:
1. Validar que os dados estão funcionando corretamente
2. Obter métricas baseline de referência

**Configuração:**
- Train/Test Split: 80/20 (stratified)
- Modelo: SVM com kernel RBF (padrão)
- Métricas: Acurácia, Precisão, Recall, F1-Score

In [30]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("=" * 80)
print("BASELINE - SVM COM KERNEL RBF")
print("=" * 80)

# Dicionário para armazenar resultados
baseline_results = []

# Lista de datasets
datasets = [
    ('Breast Cancer', X_breast_cancer, y_breast_cancer),
    ('Titanic', X_titanic, y_titanic),
    ('Water Potability', X_water_potability, y_water_potability),
    ('Employee', X_employee, y_employee),
    ('Weather', X_weather, y_weather)
]

# Para cada dataset
for name, X, y in datasets:
    print(f"\n{'-' * 80}")
    print(f"Dataset: {name}")
    print(f"{'-' * 80}")
    
    # Train/Test Split (80/20) com stratify
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.2, 
        stratify=y, 
        random_state=42
    )
    
    print(f"Treino: {X_train.shape[0]} amostras | Teste: {X_test.shape[0]} amostras")
    
    # Treinar SVM com kernel RBF
    svm = SVC(kernel='rbf', random_state=42)
    svm.fit(X_train, y_train)
    
    # Predições
    y_pred = svm.predict(X_test)
    
    # Calcular métricas
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Armazenar resultados
    baseline_results.append({
        'Dataset': name,
        'Treino': X_train.shape[0],
        'Teste': X_test.shape[0],
        'Acurácia (%)': acc * 100,
        'Precisão (%)': prec * 100,
        'Recall (%)': rec * 100,
        'F1-Score (%)': f1 * 100
    })
    
    print(f"Acurácia:  {acc*100:.2f}%")
    print(f"Precisão:  {prec*100:.2f}%")
    print(f"Recall:    {rec*100:.2f}%")
    print(f"F1-Score:  {f1*100:.2f}%")

# Criar DataFrame com resultados consolidados
print("\n" + "=" * 80)
print("TABELA RESUMO - BASELINE SVM (KERNEL RBF)")
print("=" * 80)
print()

baseline_df = pd.DataFrame(baseline_results)
# Formatar colunas de métricas com 2 casas decimais
for col in ['Acurácia (%)', 'Precisão (%)', 'Recall (%)', 'F1-Score (%)']:
    baseline_df[col] = baseline_df[col].map('{:.2f}'.format)

print(baseline_df.to_string(index=False))

print("\n" + "=" * 80)
print("✓ BASELINE CONCLUÍDO!")
print("=" * 80)

print("\n📝 Observações:")
print("  • Todos os datasets foram treinados com sucesso")
print("  • Métricas baseline disponíveis para comparação futura")
print("  • Employee: Dataset balanceado (~34% saídas) com boas métricas")
print("  • Próximo passo: Experimento RCBD com diferentes tratamentos")

BASELINE - SVM COM KERNEL RBF

--------------------------------------------------------------------------------
Dataset: Breast Cancer
--------------------------------------------------------------------------------
Treino: 455 amostras | Teste: 114 amostras
Acurácia:  97.37%
Precisão:  100.00%
Recall:    92.86%
F1-Score:  96.30%

--------------------------------------------------------------------------------
Dataset: Titanic
--------------------------------------------------------------------------------
Treino: 712 amostras | Teste: 179 amostras
Acurácia:  81.01%
Precisão:  85.71%
Recall:    60.87%
F1-Score:  71.19%

--------------------------------------------------------------------------------
Dataset: Water Potability
--------------------------------------------------------------------------------
Treino: 2620 amostras | Teste: 656 amostras
Acurácia:  67.07%
Precisão:  70.41%
Recall:    26.95%
F1-Score:  38.98%

-------------------------------------------------------------------

## 9. Otimização de Hiperparâmetros - SVM

**Objetivo:** Implementar 3 métodos de otimização de hiperparâmetros para comparação:
1. **GridSearch**: Busca exaustiva em grid definido
2. **RandomSearch**: Amostragem aleatória no espaço de busca
3. **Bayesian Optimization**: Otimização bayesiana com scikit-optimize

**Configuração:**
- Modelo: SVM com kernel RBF
- Hiperparâmetros: C e gamma
- Mesma quantidade de iterações (n_iter) para todos os métodos
- Sem cross-validation: treino em X_train, teste em X_test
- Métricas: Acurácia, Precisão, Recall, F1-Score, Tempo de execução

In [31]:
import time
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args

def grid_search_svm(X_train, y_train, X_test, y_test, n_iter=16, verbose=True):
    """
    Grid Search para otimização de hiperparâmetros do SVM.
    
    Parâmetros:
    -----------
    X_train, y_train : dados de treino
    X_test, y_test : dados de teste
    n_iter : número de combinações a testar (será ajustado para grid quadrado mais próximo)
    verbose : exibir mensagens de progresso (default=True)
    
    Retorna:
    --------
    dict com: metodo, best_params, acuracia, precisao, recall, f1_score, tempo
    """
    inicio = time.time()
    
    # Criar grid que resulte em aproximadamente n_iter combinações
    # Exemplo: n_iter=16 → 4x4 grid
    grid_size = int(np.sqrt(n_iter))
    
    # Definir ranges para C e gamma
    C_values = np.logspace(-2, 3, grid_size)  # 0.01 a 1000
    gamma_values = np.logspace(-4, 1, grid_size)  # 0.0001 a 10
    
    if verbose:
        print(f"Grid Search: testando {grid_size}x{grid_size} = {grid_size**2} combinações")
    
    best_score = 0
    best_params = {}
    best_metrics = {}
    
    # Testar todas as combinações
    for C in C_values:
        for gamma in gamma_values:
            # Treinar modelo
            model = SVC(kernel='rbf', C=C, gamma=gamma, random_state=42)
            model.fit(X_train, y_train)
            
            # Testar no conjunto de teste
            y_pred = model.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            
            # Se for o melhor, salvar
            if acc > best_score:
                best_score = acc
                best_params = {'C': C, 'gamma': gamma}
                best_metrics = {
                    'acuracia': acc,
                    'precisao': precision_score(y_test, y_pred),
                    'recall': recall_score(y_test, y_pred),
                    'f1_score': f1_score(y_test, y_pred)
                }
    
    tempo = time.time() - inicio
    
    return {
        'metodo': 'GridSearch',
        'best_params': best_params,
        'acuracia': best_metrics['acuracia'],
        'precisao': best_metrics['precisao'],
        'recall': best_metrics['recall'],
        'f1_score': best_metrics['f1_score'],
        'tempo': tempo
    }

print("✓ Função grid_search_svm() implementada!")

✓ Função grid_search_svm() implementada!


In [32]:
def random_search_svm(X_train, y_train, X_test, y_test, n_iter=16, verbose=True):
    """
    Random Search para otimização de hiperparâmetros do SVM.
    
    Parâmetros:
    -----------
    X_train, y_train : dados de treino
    X_test, y_test : dados de teste
    n_iter : número de combinações aleatórias a testar
    verbose : exibir mensagens de progresso (default=True)
    
    Retorna:
    --------
    dict com: metodo, best_params, acuracia, precisao, recall, f1_score, tempo
    """
    inicio = time.time()
    
    if verbose:
        print(f"Random Search: testando {n_iter} combinações aleatórias")
    
    best_score = 0
    best_params = {}
    best_metrics = {}
    
    np.random.seed(42)  # Para reprodutibilidade
    
    # Testar n_iter combinações aleatórias
    for i in range(n_iter):
        # Amostrar C e gamma de distribuições log-uniformes
        C = 10 ** np.random.uniform(-2, 3)  # 0.01 a 1000
        gamma = 10 ** np.random.uniform(-4, 1)  # 0.0001 a 10
        
        # Treinar modelo
        model = SVC(kernel='rbf', C=C, gamma=gamma, random_state=42)
        model.fit(X_train, y_train)
        
        # Testar no conjunto de teste
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        
        # Se for o melhor, salvar
        if acc > best_score:
            best_score = acc
            best_params = {'C': C, 'gamma': gamma}
            best_metrics = {
                'acuracia': acc,
                'precisao': precision_score(y_test, y_pred),
                'recall': recall_score(y_test, y_pred),
                'f1_score': f1_score(y_test, y_pred)
            }
    
    tempo = time.time() - inicio
    
    return {
        'metodo': 'RandomSearch',
        'best_params': best_params,
        'acuracia': best_metrics['acuracia'],
        'precisao': best_metrics['precisao'],
        'recall': best_metrics['recall'],
        'f1_score': best_metrics['f1_score'],
        'tempo': tempo
    }

print("✓ Função random_search_svm() implementada!")

✓ Função random_search_svm() implementada!


In [33]:
def bayesian_search_svm(X_train, y_train, X_test, y_test, n_iter=16, verbose=True):
    """
    Bayesian Optimization para otimização de hiperparâmetros do SVM.
    
    Parâmetros:
    -----------
    X_train, y_train : dados de treino
    X_test, y_test : dados de teste
    n_iter : número de iterações da otimização bayesiana
    verbose : exibir mensagens de progresso (default=True)
    
    Retorna:
    --------
    dict com: metodo, best_params, acuracia, precisao, recall, f1_score, tempo
    """
    inicio = time.time()
    
    if verbose:
        print(f"Bayesian Optimization: {n_iter} iterações")
    
    # Definir espaço de busca (escala logarítmica)
    space = [
        Real(1e-2, 1e3, prior='log-uniform', name='C'),
        Real(1e-4, 1e1, prior='log-uniform', name='gamma')
    ]
    
    # Variável para armazenar melhor resultado
    best_metrics = {}
    
    # Função objetivo a ser MINIMIZADA (por isso retorna -accuracy)
    @use_named_args(space)
    def objective(**params):
        C = params['C']
        gamma = params['gamma']
        
        # Treinar modelo
        model = SVC(kernel='rbf', C=C, gamma=gamma, random_state=42)
        model.fit(X_train, y_train)
        
        # Testar no conjunto de teste
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        
        # Retornar negativo da acurácia (queremos maximizar, mas gp_minimize minimiza)
        return -acc
    
    # Executar otimização bayesiana
    result = gp_minimize(
        objective,
        space,
        n_calls=n_iter,
        random_state=42,
        verbose=False
    )
    
    # Obter melhores parâmetros
    best_C = result.x[0]
    best_gamma = result.x[1]
    best_params = {'C': best_C, 'gamma': best_gamma}
    
    # Treinar modelo final com os melhores parâmetros
    final_model = SVC(kernel='rbf', C=best_C, gamma=best_gamma, random_state=42)
    final_model.fit(X_train, y_train)
    y_pred = final_model.predict(X_test)
    
    # Calcular todas as métricas
    best_metrics = {
        'acuracia': accuracy_score(y_test, y_pred),
        'precisao': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1_score': f1_score(y_test, y_pred)
    }
    
    tempo = time.time() - inicio
    
    return {
        'metodo': 'BayesianOptimization',
        'best_params': best_params,
        'acuracia': best_metrics['acuracia'],
        'precisao': best_metrics['precisao'],
        'recall': best_metrics['recall'],
        'f1_score': best_metrics['f1_score'],
        'tempo': tempo
    }

print("✓ Função bayesian_search_svm() implementada!")

✓ Função bayesian_search_svm() implementada!


### Teste das Funções de Otimização

Vamos testar as 3 funções com o dataset Breast Cancer (menor e mais rápido)

In [34]:
print("=" * 80)
print("TESTE DAS FUNÇÕES DE OTIMIZAÇÃO - BREAST CANCER")
print("=" * 80)

# Preparar dados (train/test split)
X_train, X_test, y_train, y_test = train_test_split(
    X_breast_cancer, y_breast_cancer, 
    test_size=0.2, 
    stratify=y_breast_cancer, 
    random_state=42
)

print(f"\nTreino: {X_train.shape[0]} amostras | Teste: {X_test.shape[0]} amostras")
print(f"\nTestando com n_iter=16 (4x4 grid)\n")

# Testar GridSearch
print("-" * 80)
result_grid = grid_search_svm(X_train, y_train, X_test, y_test, n_iter=16)
print(f"Método: {result_grid['metodo']}")
print(f"Melhores parâmetros: C={result_grid['best_params']['C']:.4f}, gamma={result_grid['best_params']['gamma']:.4f}")
print(f"Acurácia:  {result_grid['acuracia']*100:.2f}%")
print(f"Precisão:  {result_grid['precisao']*100:.2f}%")
print(f"Recall:    {result_grid['recall']*100:.2f}%")
print(f"F1-Score:  {result_grid['f1_score']*100:.2f}%")
print(f"Tempo:     {result_grid['tempo']:.2f}s")

# Testar RandomSearch
print("\n" + "-" * 80)
result_random = random_search_svm(X_train, y_train, X_test, y_test, n_iter=16)
print(f"Método: {result_random['metodo']}")
print(f"Melhores parâmetros: C={result_random['best_params']['C']:.4f}, gamma={result_random['best_params']['gamma']:.4f}")
print(f"Acurácia:  {result_random['acuracia']*100:.2f}%")
print(f"Precisão:  {result_random['precisao']*100:.2f}%")
print(f"Recall:    {result_random['recall']*100:.2f}%")
print(f"F1-Score:  {result_random['f1_score']*100:.2f}%")
print(f"Tempo:     {result_random['tempo']:.2f}s")

# Testar Bayesian Optimization
print("\n" + "-" * 80)
result_bayesian = bayesian_search_svm(X_train, y_train, X_test, y_test, n_iter=16)
print(f"Método: {result_bayesian['metodo']}")
print(f"Melhores parâmetros: C={result_bayesian['best_params']['C']:.4f}, gamma={result_bayesian['best_params']['gamma']:.4f}")
print(f"Acurácia:  {result_bayesian['acuracia']*100:.2f}%")
print(f"Precisão:  {result_bayesian['precisao']*100:.2f}%")
print(f"Recall:    {result_bayesian['recall']*100:.2f}%")
print(f"F1-Score:  {result_bayesian['f1_score']*100:.2f}%")
print(f"Tempo:     {result_bayesian['tempo']:.2f}s")

print("\n" + "=" * 80)
print("✓ TESTE CONCLUÍDO - TODAS AS FUNÇÕES FUNCIONANDO!")
print("=" * 80)

TESTE DAS FUNÇÕES DE OTIMIZAÇÃO - BREAST CANCER

Treino: 455 amostras | Teste: 114 amostras

Testando com n_iter=16 (4x4 grid)

--------------------------------------------------------------------------------
Grid Search: testando 4x4 = 16 combinações
Método: GridSearch
Melhores parâmetros: C=21.5443, gamma=0.0046
Acurácia:  98.25%
Precisão:  100.00%
Recall:    95.24%
F1-Score:  97.56%
Tempo:     0.30s

--------------------------------------------------------------------------------
Random Search: testando 16 combinações aleatórias
Método: RandomSearch
Melhores parâmetros: C=145.2825, gamma=0.0012
Acurácia:  96.49%
Precisão:  100.00%
Recall:    90.48%
F1-Score:  95.00%
Tempo:     0.28s

--------------------------------------------------------------------------------
Bayesian Optimization: 16 iterações
Método: BayesianOptimization
Melhores parâmetros: C=96.0981, gamma=0.0008
Acurácia:  98.25%
Precisão:  100.00%
Recall:    95.24%
F1-Score:  97.56%
Tempo:     2.00s

✓ TESTE CONCLUÍDO - TO

## 10. Experimento RCBD Completo

**Objetivo:** Executar o experimento completo comparando os 3 métodos de otimização

**Configuração:**
- **Blocos:** 5 datasets (Breast Cancer, Titanic, Water Potability, Employee, Weather)
- **Repetições:** 7 seeds diferentes (1-7) para cada dataset
- **Tratamentos:** 3 métodos de otimização (GridSearch, RandomSearch, BayesianOptimization)
- **Total de experimentos:** 5 datasets × 7 seeds × 3 métodos = **105 experimentos**

**Delineamento RCBD:**
- Datasets funcionam como blocos (controlam variabilidade)
- Métodos de otimização são os tratamentos a comparar
- Múltiplas repetições com seeds diferentes para robustez estatística

In [35]:
from tqdm.auto import tqdm
import os

# Criar dicionário com todos os datasets
datasets_dict = {
    'Breast Cancer': (X_breast_cancer, y_breast_cancer),
    'Titanic': (X_titanic, y_titanic),
    'Water Potability': (X_water_potability, y_water_potability),
    'Employee': (X_employee, y_employee),
    'Weather': (X_weather, y_weather)
}

# Definir seeds para repetições
seeds = list(range(1, 8))  # [1, 2, 3, 4, 5, 6, 7]

# Configuração do experimento
n_iter = 16  # Número de iterações para cada método

print("=" * 80)
print("CONFIGURAÇÃO DO EXPERIMENTO RCBD")
print("=" * 80)
print(f"\nNúmero de datasets (blocos): {len(datasets_dict)}")
print(f"Datasets: {list(datasets_dict.keys())}")
print(f"\nNúmero de seeds (repetições): {len(seeds)}")
print(f"Seeds: {seeds}")
print(f"\nMétodos de otimização: GridSearch, RandomSearch, BayesianOptimization")
print(f"Iterações por método: {n_iter}")
print(f"\nTotal de experimentos: {len(datasets_dict)} × {len(seeds)} × 3 = {len(datasets_dict) * len(seeds) * 3}")
print("=" * 80)

CONFIGURAÇÃO DO EXPERIMENTO RCBD

Número de datasets (blocos): 5
Datasets: ['Breast Cancer', 'Titanic', 'Water Potability', 'Employee', 'Weather']

Número de seeds (repetições): 7
Seeds: [1, 2, 3, 4, 5, 6, 7]

Métodos de otimização: GridSearch, RandomSearch, BayesianOptimization
Iterações por método: 16

Total de experimentos: 5 × 7 × 3 = 105


In [ ]:
print("\n" + "=" * 80)
print("EXECUTANDO EXPERIMENTO RCBD")
print("=" * 80)
print()

# Lista para armazenar todos os resultados
resultados = []

# Loop externo: iterar sobre datasets (blocos)
for dataset_name, (X, y) in tqdm(datasets_dict.items(), desc="Datasets", position=0):
    
    # Loop interno: iterar sobre seeds (repetições)
    for seed in tqdm(seeds, desc=f"{dataset_name}", position=1, leave=False):
        
        # 1. Fazer split train/test ESTRATIFICADO com a seed atual
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, 
            test_size=0.2, 
            stratify=y,  # Manter proporção de classes
            random_state=seed
        )
        
        # 2. GridSearch
        result_grid = grid_search_svm(X_train, y_train, X_test, y_test, n_iter=n_iter, verbose=False)
        result_grid['dataset'] = dataset_name
        result_grid['seed'] = seed
        resultados.append(result_grid)
        
        # 3. RandomSearch
        result_random = random_search_svm(X_train, y_train, X_test, y_test, n_iter=n_iter, verbose=False)
        result_random['dataset'] = dataset_name
        result_random['seed'] = seed
        resultados.append(result_random)
        
        # 4. BayesianOptimization
        result_bayesian = bayesian_search_svm(X_train, y_train, X_test, y_test, n_iter=n_iter, verbose=False)
        result_bayesian['dataset'] = dataset_name
        result_bayesian['seed'] = seed
        resultados.append(result_bayesian)

print("\n" + "=" * 80)
print("✓ EXPERIMENTO CONCLUÍDO!")
print("=" * 80)
print(f"\nTotal de resultados coletados: {len(resultados)}")
print("\n" + "=" * 80)
print("CONSOLIDANDO RESULTADOS")
print("=" * 80)

# Converter lista de dicts para DataFrame
df_resultados = pd.DataFrame(resultados)

# Reordenar colunas para melhor visualização
colunas_ordenadas = ['dataset', 'seed', 'metodo', 'acuracia', 'precisao', 'recall', 'f1_score', 'tempo', 'best_params']
df_resultados = df_resultados[colunas_ordenadas]

# Mostrar informações básicas
print(f"\nShape do DataFrame: {df_resultados.shape}")
print(f"Colunas: {list(df_resultados.columns)}")

# Mostrar primeiras linhas
print("\n" + "-" * 80)
print("PRIMEIRAS 10 LINHAS:")
print("-" * 80)
print(df_resultados.head(10).to_string(index=False))

# Estatísticas descritivas por método
print("\n" + "=" * 80)
print("ESTATÍSTICAS DESCRITIVAS POR MÉTODO")
print("=" * 80)

for metodo in ['GridSearch', 'RandomSearch', 'BayesianOptimization']:
    df_metodo = df_resultados[df_resultados['metodo'] == metodo]
    print(f"\n{metodo}:")
    print("-" * 80)
    print(f"Acurácia:  média={df_metodo['acuracia'].mean():.4f}, std={df_metodo['acuracia'].std():.4f}")
    print(f"Precisão:  média={df_metodo['precisao'].mean():.4f}, std={df_metodo['precisao'].std():.4f}")
    print(f"Recall:    média={df_metodo['recall'].mean():.4f}, std={df_metodo['recall'].std():.4f}")
    print(f"F1-Score:  média={df_metodo['f1_score'].mean():.4f}, std={df_metodo['f1_score'].std():.4f}")
    print(f"Tempo(s):  média={df_metodo['tempo'].mean():.2f}s, std={df_metodo['tempo'].std():.2f}s")

print("\n" + "=" * 80)
print("✓ RESULTADOS CONSOLIDADOS!")
print("=" * 80)


EXECUTANDO EXPERIMENTO RCBD



Datasets:   0%|          | 0/5 [00:00<?, ?it/s]

Breast Cancer:   0%|          | 0/7 [00:00<?, ?it/s]

Titanic:   0%|          | 0/7 [00:00<?, ?it/s]

Water Potability:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
print("\n" + "=" * 80)
print("SALVANDO RESULTADOS")
print("=" * 80)

# Criar diretório results se não existir
results_dir = '../results'
os.makedirs(results_dir, exist_ok=True)

# Salvar DataFrame em CSV
output_file = os.path.join(results_dir, 'experimento_rcbd_resultados.csv')
df_resultados.to_csv(output_file, index=False)

print(f"\n✓ Resultados salvos em: {output_file}")
print(f"  Total de linhas: {len(df_resultados)}")
print(f"  Colunas: {', '.join(df_resultados.columns)}")

# Também salvar versão JSON dos best_params separadamente para facilitar análise
import json

# Expandir best_params em colunas separadas
df_resultados_expandido = df_resultados.copy()
df_resultados_expandido['C'] = df_resultados_expandido['best_params'].apply(lambda x: x['C'])
df_resultados_expandido['gamma'] = df_resultados_expandido['best_params'].apply(lambda x: x['gamma'])
df_resultados_expandido = df_resultados_expandido.drop(columns=['best_params'])

# Salvar versão expandida
output_file_expandido = os.path.join(results_dir, 'experimento_rcbd_resultados_expandido.csv')
df_resultados_expandido.to_csv(output_file_expandido, index=False)

print(f"\n✓ Versão expandida salva em: {output_file_expandido}")
print(f"  (best_params separado em colunas C e gamma)")

print("\n" + "=" * 80)
print("✓ EXPERIMENTO RCBD COMPLETO - TODOS OS DADOS SALVOS!")
print("=" * 80)

print("\n📊 Próximos passos:")
print("  1. Análise exploratória dos resultados")
print("  2. ANOVA para comparar métodos de otimização")
print("  3. Testes post-hoc se ANOVA indicar diferenças significativas")
print("  4. Visualizações (boxplots, gráficos de interação)")
print("  5. Conclusões e recomendações")